<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_intelligence_tfq_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq tensorflow_quantum

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import argparse
import json
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq


def set_seed(seed: int = 42):
    np.random.seed(seed)
    tf.random.set_seed(seed)


def build_parametric_circuit(n_qubits: int = 4):
    """Return (qubits, circuit, symbols, readout_ops)."""
    qubits = cirq.GridQubit.rect(1, n_qubits)
    symbols = list(sympy.symbols(f"x0:{n_qubits}"))

    circuit = cirq.Circuit()
    # Feature encoding + light entanglement
    for i, q in enumerate(qubits):
        circuit.append(cirq.H(q))
        circuit.append(cirq.rz(symbols[i])(q))
        circuit.append(cirq.rx(symbols[i] / 2.0)(q))
    for i in range(n_qubits - 1):
        circuit.append(cirq.CZ(qubits[i], qubits[i + 1]))
    # Close a ring for 4 qubits
    if n_qubits > 2:
        circuit.append(cirq.CZ(qubits[-1], qubits[0]))

    readout_ops = [cirq.Z(q) for q in qubits]  # 4 expectation values
    return qubits, circuit, symbols, readout_ops


def make_model(n_qubits: int = 4, hidden: int = 16, lr: float = 1e-3, n_classes: int = 4):
    qubits, model_circuit, symbols, readouts = build_parametric_circuit(n_qubits)

    # TFQ expects a string tensor of circuits, and a float tensor of symbol values
    circuits_in = tf.keras.Input(shape=(), dtype=tf.dtypes.string, name="circuits")
    values_in = tf.keras.Input(shape=(len(symbols),), dtype=tf.float32, name="symbol_values")

    pqc_out = tfq.layers.PQC(model_circuit, operators=readouts)([circuits_in, values_in])
    x = tf.keras.layers.Dense(hidden, activation="relu")(pqc_out)
    x = tf.keras.layers.Dense(hidden, activation="relu")(x)
    logits = tf.keras.layers.Dense(n_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs=[circuits_in, values_in], outputs=logits)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model, symbols


def generate_data(n: int = 2000, n_features: int = 4, seed: int = 42):
    """Features in [0, 2π]; labels = argmax(feature groups) => learnable pattern."""
    rng = np.random.default_rng(seed)
    X = rng.random((n, n_features), dtype=np.float32) * (2.0 * np.pi)

    # Deterministic labels: choose the index of the strongest (wrapped) signal
    # You can tailor this to your use case.
    y = np.argmax(
        np.stack([
            np.sin(X[:, 0]) + np.cos(X[:, 1]),
            np.sin(X[:, 1]) + np.cos(X[:, 2]),
            np.sin(X[:, 2]) + np.cos(X[:, 3]),
            np.sin(X[:, 3]) + np.cos(X[:, 0]),
        ], axis=1),
        axis=0
    ).astype(np.int32)
    return X, y


def to_tfq_inputs(n_samples: int, base_circuit: cirq.Circuit):
    """Create a batch of identical data circuits (no per-example ops)."""
    circuits = tfq.convert_to_tensor([base_circuit] * n_samples)
    return circuits


def sanitize_argv(argv):
    """Remove notebook-injected args like '-f ...kernel.json'."""
    cleaned = []
    skip = False
    for i, a in enumerate(argv):
        if skip:
            skip = False
            continue
        if a == "-f":
            skip = True
            continue
        if a.endswith(".json") and "jupyter/runtime/kernel" in a:
            continue
        cleaned.append(a)
    return cleaned


def parse_args():
    p = argparse.ArgumentParser(description="TFQ classifier: circuits + PQC + Dense layers (notebook-safe).")
    p.add_argument("--samples", type=int, default=2000)
    p.add_argument("--epochs", type=int, default=20)
    p.add_argument("--batch-size", type=int, default=64)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--lr", type=float, default=1e-3)
    p.add_argument("--hidden", type=int, default=32)
    p.add_argument("--val-ratio", type=float, default=0.2)
    p.add_argument("--outdir", type=str, default="tfq_runs")
    args, _ = p.parse_known_args(sanitize_argv(sys.argv[1:]))
    return args


def main_cli():
    args = parse_args()
    os.makedirs(args.outdir, exist_ok=True)
    set_seed(args.seed)

    # Build model
    model, symbols = make_model(hidden=args.hidden, lr=args.lr)
    n_features = len(symbols)

    # Data
    X, y = generate_data(n=args.samples, n_features=n_features, seed=args.seed)
    n_val = int(args.val_ratio * args.samples)
    X_train, y_train = X[:-n_val], y[:-n_val]
    X_val, y_val = X[-n_val:], y[-n_val:]

    # TFQ circuit inputs (same base circuit for all examples)
    _, base_circuit, _, _ = build_parametric_circuit(n_qubits=n_features)
    circuits_train = to_tfq_inputs(len(X_train), base_circuit)
    circuits_val = to_tfq_inputs(len(X_val), base_circuit)

    # Train
    history = model.fit(
        x=[circuits_train, X_train],
        y=y_train,
        validation_data=([circuits_val, X_val], y_val),
        epochs=args.epochs,
        batch_size=args.batch_size,
        verbose=1,
    )

    # Evaluate
    val_loss, val_acc = model.evaluate([circuits_val, X_val], y_val, verbose=0)
    print(f"Validation — loss: {val_loss:.4f}, acc: {val_acc:.4f}")

    # Predict on a small batch
    X_test = np.random.random((8, n_features)).astype(np.float32) * (2.0 * np.pi)
    circuits_test = to_tfq_inputs(len(X_test), base_circuit)
    probs = model.predict([circuits_test, X_test], verbose=0)
    preds = np.argmax(probs, axis=1)
    print("Sample predictions:", preds.tolist())

    # Save artifacts
    summary = {
        "config": vars(args),
        "val_loss": float(val_loss),
        "val_acc": float(val_acc),
    }
    with open(os.path.join(args.outdir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    model.save(os.path.join(args.outdir, "model.keras"))
    print(f"Saved artifacts to {args.outdir}")


if __name__ == "__main__":
    main_cli()